# Proyección de matrícula escolar de múltiples escuelas primarias modeladas como series de tiempo.

#### Importante:
La presente libreta es una herramienta de documentación para la elaboración de la tesis, no pretende ser la implementación final del sistema de proyección de matrícula.
Para ejecutar una celda, dar click en ella y presionar ctrl + enter.

Instalar paquetes necesarios en caso de que no se cuente con ellos. Saltar esta celda si se ejecuta desde Binder.

In [ ]:
# Recargar página luego de ejecutar esta celda por primera vez
!pip install "ipywidgets>=7.2"
!pip install sklearn
!pip install tensorflow
!pip install pandas
!pip install numpy
!pip install bqplot
!pip install datapane
!jupyter nbextension enable --py --sys-prefix bqplot

Importar todos los paquetes necesarios para esta libreta.

In [1]:
import pandas as pd
import numpy as np
import bqplot.pyplot as plt
import tensorflow as tf
from IPython.display import display, HTML, IFrame
import ipywidgets as widgets
import altair as alt
import datapane as dp

from bqplot import (ColorScale, DateColorScale, OrdinalColorScale, 
                    LinearScale, Tooltip)
from ipywidgets import interact, interactive, fixed, interact_manual

# Evaluación de modelos

En las siguientes celdas se presenta una evaluación de los modelos y después se continúa con la descripción de las métricas.

In [2]:
# Manejo de modulos
import sys, os
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

# Importar las funciones de los modelos
#from Metodos.TimeSeries import time_series_prediction
from Metodos.DNN import load_model, dnn_predict, dnn_predict_by_year
from Metodos.TimeSeriesPrediction import fixed_partitioning_predict
from Metodos.LinearRegression import linear_regression_predict, base_linear_regression
from Metodos.NaiveForecasting import naive_forecasting_predict
from Metodos.AutoARIMA import auto_arima_predict
from Metodos.RNN import rnn_predict
from Metodos.OnFlyCNN import custom_cnn_predict
from Metodos.DifferencedDNN import differenced_dnn

# Importar interface de modelos
from Evaluation import Model, TestResult

# Crear modelos

model_dict = {
    'Fix. part. time series' : Model(fixed_partitioning_predict),
    'Fix. part. linear regression' : Model(linear_regression_predict),
    'Deep Neural Networks' : Model(dnn_predict, args = dict(cached_model = load_model('ValidacionPrimarias'))),
    'Linear regression' : Model(base_linear_regression),
    'Naive forecasting' : Model(naive_forecasting_predict),
    'Auto ARIMA' : Model(auto_arima_predict),
    'Differenced DNN' : Model(differenced_dnn, args = dict(window_len = 5)),
    'Recurrent Neural Networks' : Model(rnn_predict, args = dict(window_size = 5)),
    'Convolutional Neural Networks' : Model(custom_cnn_predict)
}

# Crear etiquetas
label_dict = {
    'Fix. part. time series' : 'Fixed partitioning time series',
    'Fix. part. linear regression' : 'Fixed partitioning linear regression',
    'Deep Neural Networks' : 'Redes neuronales',
    'Linear regression' : 'Regresión lineal',
    'Naive forecasting' : 'Naïve forecasting',
    'Auto ARIMA:' : 'Auto ARIMA',
    'Differenced DNN' : 'Differenced DNN',
    'Recurrent Neural Networks' : 'Recurrent Neural Networks',
    'Convolutional Neural Networks' : 'Convolutional Neural Networks',
    'Primarias públicas y privadas' : 'PrimariasCompletas',
    'Primarias públicas' : 'PrimariasPublicas',
    'Primarias privadas' : 'PrimariasPrivadas',
    'Muestra' : 'Muestra',
    'Mediana 1' : 'Mediana1Primarias',
    'Mediana 2' : 'Mediana2Primarias'
}

Modelo ValidacionPrimarias cargado en memoria


In [3]:
# Seleccionar conjunto de datos, método de predicción y número de años a predecir
out = widgets.Output()
evaluacion_dict = None

@interact(
    modelo = widgets.RadioButtons(
        options = [
            'Fix. part. time series', 
            'Fix. part. linear regression', 
            'Deep Neural Networks',
            'Linear regression',
            'Naive forecasting',
            'Auto ARIMA',
            'Differenced DNN',
            'Recurrent Neural Networks',
            'Convolutional Neural Networks'
        ],
        description='Modelo',
        disabled=False
    ),
    conjunto = widgets.RadioButtons(
        options=[
            'Primarias públicas y privadas',
            'Primarias públicas',
            'Primarias privadas',
            'Muestra',
            'Mediana 1',
            'Mediana 2'
        ],
        description='Conjunto',
        disabled=False
    ),
    anios = widgets.IntSlider(
        value=3,
        min=1,
        max=5,
        step=1,
        description='Años',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d'
    )
)
def preparar_evaluacion(modelo, conjunto, anios) :
    global evaluacion_dict
    evaluacion_dict = {
        'modelo' : modelo,
        'conjunto' : conjunto,
        'anios' : anios
    }

interactive(children=(RadioButtons(description='Modelo', options=('Fix. part. time series', 'Fix. part. linear…

In [4]:
# Calcular resultados de la prueba
from MyUtilities import LoadEvaluationPlot
from MyUtilities import LoadHTMLTable

modelo = evaluacion_dict['modelo']
conjunto = evaluacion_dict['conjunto']
prediction_size = evaluacion_dict['anios']

model = model_dict[modelo]
result = model.test_set(label_dict[conjunto], prediction_size, "GruposPrimaria")

LoadEvaluationPlot(result, prediction_size, conjunto, modelo)

out_1 = widgets.Output()
with out_1:
    out_1.clear_output(True)
    plt.show()
    display(HTML(LoadHTMLTable(result.metricas, modelo)))
    
out_1

(4, 2914)


KeyboardInterrupt: 

## Métricas de evaluación

A continuación se presenta una descripción de las métricas seleccionadas.

#### Notas importantes:

Las métricas son calculadas por año de forma independiente.

$\hat{Y}:$ vector de predicción del conjunto de escuelas.

$Y:$ vector de datos reales (alumnos inscritos) del conjunto de escuelas.

El error de una predicción se calcula como $E_i = Y_i - \hat{Y}_i$, y puede ser explicado gráficamente en la siguiente figura:

![](DefinicionError.png)

Cada punto representa una predicción, la distancia vertical entre cada punto y la línea base de predicción es el error en la predicción. Los errores son negativos si se encuentran por debajo de la línea base de predicción y son positivos si se encuentran por encima de la línea base de predicción. La línea base de predicción representa la predicción ideal sobre la que deberían estar todas las predicciones en un modelo perfecto.

### Mean absolute error

Métrica que representa el promedio de error de todas las predicciones realizadas. Otorga la ventaja de representar el error en la misma escala en la que se encuentran los datos.

Cálculo:

$$MAE = \frac{1}{n} \sum_{i = 1}^{n} |\hat{Y}_i - Y_i |$$

### Root mean squared error

Similar al MAE esta métrica representa el error en una escala cercana a la escala de los datos pero que funciona mejor cuando existen variaciones en la escala de los datos.

Cálculo:

$$RMSE = \sqrt{ \frac{1}{n} \sum_{i = 1}^{n} (\hat{Y}_i - Y_i) ^ {2}}$$

### Mean absolute percentage error

Métrica estándard que representa el porcentage de error promedio de todas las predicciones realizadas.

Cálculo:

$$MAPE = \frac{1}{n} \sum_{i = 1} ^ n \frac{|\hat{Y}_i - Y_i|}{Y_i}$$

### Probabilidad de riesgo

La probabilidad de riesgo es la única métrica no estándard presente en la evaluación de los modelos. Representa la probabilidad de tomar una proyección de matrícula cuyo error supere al número de alumnos por grupo en dicha escuela. Este tipo de proyecciones son peligrosas ya que en caso de incurrir en una de ellas, se estará tomando una decisión que pueda perjudicar los recursos involucrados, por ejemplo, predecir con un error de 40 alumnos en una escuela en la que el promedio de alumnos por grupo es de 20 alumnos puede conducirnos a casos en los que se inviertan recursos en la creación de dos nuevos grupos, o casos en los que sea necesaria la creación de dos nuevos grupos en un momento tardío del periodo de clases.

Cálculo de la probabilidad de riesgo $PR$:

$$PR = \frac{1}{n} \sum_{i = 1} ^ {n} f(i)$$

$$
f(i)=
\begin{cases}
1 \Leftrightarrow |\hat{Y}_i - Y_i| \geq \frac{Y_i}{G_i} \\
0 \Leftrightarrow |\hat{Y}_i - Y_i| < \frac{Y_i}{G_i} \\
\end{cases}
$$ 

$$PR \in \Bbb R, f(i) : 1 \leq i \leq n \rightarrow [0, 1]$$

donde $G_i$ representa el número de grupos en la escuela $i$.

# Proyección de matrícula en una sola escuela

En la siguiente celda se evalúa la proyección de matrícula en una escuela dados su clave de centro de trabajo, modelo de predicción y años a predecir.

#### Importante: solo se aceptan cct de escuelas primarias

In [5]:
# Introduce el CCT, los años a predecir (5 máximo) y el método de predicción
out = widgets.Output()
reporte_dict = None

@interact(cct = 'Introduce cct', 
          anios = widgets.IntSlider(min=1, max=5, step=1, value=3, description = 'Años'), 
          modelo = widgets.RadioButtons(
              options = [
                  'Fix. part. time series',
                  'Fix. part. linear regression', 
                  'Deep Neural Networks',
                  'Linear regression',
                  'Naive forecasting',
                  'Auto ARIMA',
                  'Differenced DNN',
                  'Recurrent Neural Networks',
                  'Convolutional Neural Networks'
              ],
              description='Modelo',
              disabled=False
))
def prediccion(cct, anios, modelo) :
    global model_dict, reporte_dict
    model = model_dict[modelo]
    
    dataset = pd.read_csv(os.path.join(
        os.path.abspath(os.pardir), 
        'Datasets/', 
        'PrimariasCompletas.csv'
    ))
    unique_index = pd.Index(list(dataset['cct']))
    if cct in unique_index :
        index = unique_index.get_loc(cct)
        print('Se encontró el cct')
    else :
        print('No se encontró el cct')
        return
    
    # Quitar el cct
    row = np.array(dataset.loc[index][1:])
    
    # Separamos los últimos 5 años
    X = row[: -model.TEST_SIZE]

    if anios == model.TEST_SIZE :
        Y = row[-model.TEST_SIZE :]
    else :
        Y = row[-model.TEST_SIZE : -(model.TEST_SIZE - anios)]
    prediccion = model.predict(X, anios)
    
    list_X = list(X)
    list_Y = list(Y)
    list_predict = list(prediccion)
    
    # Preparar reporte
    reporte_dict = {
        'title' : 'Predicción de matrícula escolar en la escuela %s utilizando %s' % (cct, modelo),
        'X' : X,
        'Y' : Y,
        'prediccion' : prediccion,
        'anios' : anios
    }
    
    # Mostrar gráfica
    with out:
        out.clear_output(True)
        
        axes_options = {'x': {'label': 'Año'},'y': {'label': 'Alumnos'}}
        
        fig = plt.figure(title = 'Predicción de matrícula escolar en la escuela %s utilizando %s' % (cct, modelo), legend_location='top-left')
        plt.plot(x = [1998 + i for i in range(len(X))], y = list_X, colors = ['red'], axes_options=axes_options, labels = ['Datos de entrenamiento'], display_legend = True, marker = 'circle')
        plt.plot(x = [1997 + len(X) + i  for i in range(len(Y) + 1)], y = (list_X + list_Y)[-(anios + 1):], labels = ['Datos de prueba'], display_legend = True, marker = 'circle')
        plt.plot(x = [1997 + len(X) + i  for i in range(len(Y) + 1)], y = (list_X + list_predict)[-(anios + 1):], colors = ['green'], labels = ['Datos predecidos'], display_legend = True, marker = 'circle')
        
        plt.show()
out

interactive(children=(Text(value='Introduce cct', description='cct'), IntSlider(value=3, description='Años', m…

Output()

## Compartir reporte en Datapane

En la siguientes celdas se pueden crear reportes de la proyección hecha anteriormente y compartirlas públicamente.

In [ ]:
# Escribir aquí el token de acceso de Datapane
# Puedes encontrarlo en: https://datapane.com/home/

!datapane login --server=https://datapane.com/ --token=TOKEN

In [ ]:
from MyUtilities import LoadReport

# Crear reporte
reporte = LoadReport(**reporte_dict)

# Ver reporte en esta celda de Jupyter
reporte.preview()

# Alternativa: guardar reporte en disco y mostrar IFrame en Jupyter
#reporte.save(path='Reporte.html', open = True)
#display(IFrame(src = 'Reporte.html', width = '100%', height = '540px'))

# Publicar
reporte.publish(name='Reporte', open=True, visibility='PUBLIC')